# Read data from NVDB and convert to RDF for comparision with Overture Maps

In [ ]:
import sys, datetime
from rdflib import Graph, Namespace

# Parameters
rootPath = "C:\+Data\\GitHub\\jetgeo\\OM2ANY"
nvdb_otl = "C:\\Data\\GitHub\\jetgeo\\OM2ANY\\OWL\\nvdb\\nvdb-owl.ttl"

